In [4]:
import tkinter as tk
from tkinter import ttk
import mysql.connector

class IPLStatsViewer:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("IPL Stats Viewer")
        self.root.attributes("-fullscreen", True)
        self.create_widgets()
        self.root.mainloop()

    def create_widgets(self):
        image_path = "download__1_-removebg-preview.png"
        image = tk.PhotoImage(file=image_path)
        logo_label = tk.Label(self.root, image=image)
        logo_label.image = image
        logo_label.pack(pady=20)

        label_title = tk.Label(self.root, text="IPL Statistics", font=("Helvetica", 36, "bold"))
        label_title.pack(side=tk.TOP, pady=4)

        button_frame = ttk.Frame(self.root)
        button_frame.pack()

        stats_buttons = [("Show Player Info", "player_info"), ("Best Batting Stats", "most_runs"),
                         ("Best Bowling Stats", "most_wickets"), ("Most Catches Stats", "most_catches")]
        for text, table_name in stats_buttons:
            self.create_button(button_frame, text, table_name)

        label_search = tk.Label(self.root, text="Search by Player:", font=("Helvetica", 18))
        label_search.pack(pady=20)

        self.search_var = tk.StringVar()
        self.search_entry = tk.Entry(self.root, textvariable=self.search_var, font=("Helvetica", 14))
        self.search_entry.pack(pady=5)

        self.search_button = tk.Button(self.root, text="Search", font=("Helvetica", 14), command=self.search_player)
        self.search_button.pack(pady=8)

        label_changes = tk.Label(self.root, text="Make Changes in The Player's Info Table Below : ", font=("Helvetica", 22))
        label_changes.pack(pady=18)

        self.player_get_entry = tk.Entry(self.root, font=("Helvetica", 14))
        self.player_get_entry.pack(pady=8)
        self.player_get_entry.insert(tk.END, "*Old Value*")

        self.player_data_entry = tk.Entry(self.root, font=("Helvetica", 14))
        self.player_data_entry.pack(pady=8)
        self.player_data_entry.insert(tk.END, "*New Value*")

        self.delete_button = tk.Button(self.root, text="Delete", font=("Helvetica", 14), command=self.delete_player)
        self.delete_button.pack(pady=7)

        self.update_button = tk.Button(self.root, text="Update", font=("Helvetica", 14), command=self.update_player)
        self.update_button.pack(pady=7)

        self.create_button = tk.Button(self.root, text="Create_Player_Info", font=("Helvetica", 14), command=self.create_player_info)
        self.create_button.pack(pady=7)

        self.close_button = tk.Button(self.root, text="Close", font=("Helvetica", 14), command=self.root.destroy)
        self.close_button.pack(pady=18)

    def create_button(self, parent, text, table_name):
        button_styles = {"font": ("Helvetica", 14), "width": 15, "height": 3}
        button = tk.Button(parent, text=text, **button_styles, command=lambda: self.show_stats(table_name, text))
        button.pack(side=tk.LEFT, padx=5, pady=5)

    def show_stats(self, table_name, stats_title):
        conn = mysql.connector.connect(host="localhost", database="harsh", user="root", password="6063")
        cursor = conn.cursor(dictionary=True)
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        players = cursor.fetchall()
        conn.close()
        self.display_stats_window(stats_title, players)

    def display_stats_window(self, stats_title, players):
        stats_window = tk.Toplevel(self.root)
        stats_window.title(stats_title)
        stats_frame = ttk.Frame(stats_window)
        stats_frame.pack(fill=tk.BOTH, expand=1)
        stats_heading = ttk.Label(stats_frame, text=stats_title, font=('Helvetica', 16, 'bold'))
        stats_heading.pack(pady=10)
        headers = list(players[0].keys())
        self.display_table(stats_frame, headers, players)

    def search_player(self):
        player_name = self.search_var.get()
        self.execute_query(f"SELECT pi.Player_Name,pi.Team,pi.Nationality,b.Span,b.Matches,b.runs,b.SR,b.hs,b.Average FROM player_info AS pi JOIN most_runs AS b ON pi.player_name = b.player_name where pi.Player_Name LIKE '%{player_name}%'")

    def create_player_info(self):
        player_info = self.player_data_entry.get()
        player_info_list = player_info.split(',')
        player_name, player_team, player_nationality, player_type = [info.strip() for info in player_info_list]
        self.execute_query(f"INSERT INTO Player_Info (Player_Name, Team, Nationality, Player_Type) VALUES ('{player_name}', '{player_team}', '{player_nationality}', '{player_type}')")

    def delete_player(self):
        player_name = self.player_data_entry.get()
        self.execute_query(f"DELETE FROM Player_Info WHERE Player_Name = '{player_name}'")

    def update_player(self):
        player_name = self.player_get_entry.get()
        player_data = self.player_data_entry.get()
        self.execute_query(f"UPDATE Player_Info SET Player_Name = '{player_data}' WHERE Player_Name = '{player_name}'")

    def execute_query(self, query):
     conn = mysql.connector.connect(host="localhost", database="harsh", user="root", password="6063")
     cursor = conn.cursor(dictionary=True)
     cursor.execute(query)

     result = cursor.fetchall() if query.strip().upper().startswith("SELECT") else None
     conn.commit() if not result else None

     cursor.close()
     conn.close()

     if result:
        self.display_search_results(result)
        print(f"{len(result)} record(s) found.")
     else:
        print(f"{cursor.rowcount} record(s) updated.")


    def display_search_results(self, result):
        search_results_window = tk.Toplevel(self.root)
        search_results_window.title("Search Results")
        results_frame = ttk.Frame(search_results_window)
        results_frame.pack(fill=tk.BOTH, expand=1)
        headers = list(result[0].keys())
        self.display_table(results_frame, headers, result)

    def display_table(self, parent, headers, players):
     header_frame = ttk.Frame(parent)
     header_frame.pack()
     [ttk.Label(header_frame, text=header, font=('Helvetica', 12, 'bold')).pack(side=tk.LEFT, padx=20, pady=8) for header in headers]

     [self.display_player_data(ttk.Frame(parent), player).pack() for player in players]

    def display_player_data(self, data_frame, player):
     [ttk.Label(data_frame, text=value, font=('Helvetica', 12)).pack(side=tk.LEFT, padx=25, pady=8) for value in player.values()]
     return data_frame 
if __name__ == "__main__":
    app = IPLStatsViewer()


1 record(s) found.
1 record(s) updated.
1 record(s) updated.
1 record(s) updated.
